# Tutorial loading and exploring LabelMe dataset

## Basic imports

In [1]:
import numpy as np
from pathlib import Path

In [2]:
DIR = Path().cwd()
DIRlabelme = (DIR / ".." / "datasets" / "labelme").resolve()
DIR_module = DIRlabelme / "labelme.py"
print(DIRlabelme)

/home/tlefort/Documents/peerannot/peerannot/datasets/labelme


## Install dataset

In [3]:
! peerannot install $DIR_module

Replace is False and data exists, so doing nothing. Use replace=True to re-download the data.
Loading data folders at /home/tlefort/Documents/peerannot/peerannot/datasets/labelme
Created:
- train: /home/tlefort/Documents/peerannot/peerannot/datasets/labelme/train
- val: /home/tlefort/Documents/peerannot/peerannot/datasets/labelme/val
- test: /home/tlefort/Documents/peerannot/peerannot/datasets/labelme/test
Traceback (most recent call last):
  File "/home/tlefort/condaenvs/phd/bin/peerannot", line 33, in <module>
    sys.exit(load_entry_point('PeerAnnot', 'console_scripts', 'peerannot')())
  File "/home/tlefort/condaenvs/phd/lib/python3.9/site-packages/click/core.py", line 1134, in __call__
    return self.main(*args, **kwargs)
  File "/home/tlefort/condaenvs/phd/lib/python3.9/site-packages/click/core.py", line 1059, in main
    rv = self.invoke(ctx)
  File "/home/tlefort/condaenvs/phd/lib/python3.9/site-packages/click/core.py", line 1665, in invoke
    return _process_result(sub_ctx.co

# Majority vote

In [10]:
! peerannot aggregate $DIRlabelme -s MV

Running aggregation mv with options {}
Aggregated labels stored at /home/tlefort/Documents/peerannot/peerannot/datasets/labelme/labels/labels_labelme_mv.npy with shape (1000,)


# Naive Soft labelling

In [12]:
! peerannot aggregate $DIRlabelme -s NaiveSoft

Running aggregation naivesoft with options {}
Aggregated labels stored at /home/tlefort/Documents/peerannot/peerannot/datasets/labelme/labels/labels_labelme_naivesoft.npy with shape (1000, 8)


# Loading datasets

The majority voting in case of equality of votes returns one of the possible classes with equal probabilities (the naive soft aggregation computes the accuracy on aggregation with a simple `np.argmax` and thus does not sample the equalities).
This can lead to small differences in aggregation accuracy in practice.

In [11]:
from peerannot.runners.train import load_all_data
labels_path = DIRlabelme / "labels" / "labels_labelme_mv.npy"
trainset, valset, testset = load_all_data(DIRlabelme, labels_path, path_remove=None, labels=labels_path)

Loading datasets
Accuracy on aggregation: 76.100%


In [13]:
labels_path = DIRlabelme / "labels" / "labels_labelme_naivesoft.npy"
trainset, valset, testset = load_all_data(DIRlabelme, labels_path, path_remove=None, labels=labels_path)

Loading datasets
Accuracy on aggregation: 76.900%


# WAUM stacked identification

In [4]:
path_votes = DIRlabelme / "answers.json"

In [5]:
! peerannot identify $DIRlabelme -K 8 --method WAUMstacked --labels $path_votes\
    --model vgg16_bn --n-epochs 2 --lr=0.1\
    --maxiter-DS=50 --pretrained

Running the following configuration:
----------
- Data at /home/tlefort/Documents/peerannot/peerannot/datasets/labelme
- number of classes: 8
- labels: /home/tlefort/Documents/peerannot/peerannot/datasets/labelme/answers.json
- model: vgg16_bn
- n_epochs: 2
- lr: 0.1
- maxiter_ds: 50
- pretrained: True
- use_pleiss: False
- alpha: 0.01
- n_params: 3072
- momentum: 0.9
- decay: 0.0005
- img_size: 224
----------
Train set: 2547 tasks
Using cache found in /home/tlefort/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/tlefort/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/tlefort/.cache/torch/hub/pytorch_vision_main
Successfully loaded vgg16_bn with n_classes=8
Running identification with method: WAUMstacked
epoch:   0%|                                              | 0/2 [00:03<?, ?it/s]
Traceback (most recent call last):
  File "/home/tlefort/condaenvs/phd/bin/peerannot", line 33, in <module>
    sys.exit(load_entry_point('PeerAnnot', 'console_scripts', 'pe